<a href="https://colab.research.google.com/github/Dongrisajid/HarvestAI-Empowering-Farmers-with-Advanced-AI-Solutionsh/blob/main/project_implementation_for_real_time_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import RPi.GPIO as GPIO
import time
import dht11
import requests

# Setup GPIO mode
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Setup GPIO pins
moisture_pin = 17     # Soil moisture sensor pin
relay_pin = 27        # Relay control pin for water pump
rain_sensor_pin = 22  # Rain sensor pin
dht11_pin = 4         # DHT11 sensor pin

# Setup GPIO for relay and sensors
GPIO.setup(moisture_pin, GPIO.IN)
GPIO.setup(relay_pin, GPIO.OUT)
GPIO.setup(rain_sensor_pin, GPIO.IN)
GPIO.setup(dht11_pin, GPIO.IN)

# ThingSpeak settings
thingSpeakAPIKey = 'YOUR_THINGSPEAK_API_KEY'
thingSpeakURL = f"https://api.thingspeak.com/update?api_key={thingSpeakAPIKey}"

# Function to read DHT11 (temperature and humidity)
def read_dht11():
    instance = dht11.DHT11(pin=dht11_pin)
    result = instance.read()
    if result.is_valid():
        temperature = result.temperature
        humidity = result.humidity
        return temperature, humidity
    return None, None

# Function to check soil moisture
def read_soil_moisture():
    return GPIO.input(moisture_pin)

# Function to check rain status
def read_rain_sensor():
    return GPIO.input(rain_sensor_pin)

# Function to send data to ThingSpeak
def send_data_to_thingspeak(temperature, humidity, moisture, rain_status):
    payload = f"&field1={temperature}&field2={humidity}&field3={moisture}&field4={rain_status}"
    response = requests.get(thingSpeakURL + payload)
    print(response.text)

# Main loop to collect data and control irrigation
try:
    while True:
        # Read sensors
        temperature, humidity = read_dht11()
        soil_moisture = read_soil_moisture()
        rain_status = read_rain_sensor()

        # Control pump based on moisture and rain status
        if soil_moisture == 0 and rain_status == 0:  # Low moisture and no rain
            GPIO.output(relay_pin, GPIO.HIGH)  # Turn on pump
        else:
            GPIO.output(relay_pin, GPIO.LOW)   # Turn off pump

        # Send data to ThingSpeak
        if temperature is not None and humidity is not None:
            send_data_to_thingspeak(temperature, humidity, soil_moisture, rain_status)

        # Wait before next reading
        time.sleep(60)  # Delay for 1 minute (adjustable)

except KeyboardInterrupt:
    print("Program stopped manually.")
    GPIO.cleanup()


ModuleNotFoundError: No module named 'RPi'